In [1]:
!pip install pdf2image pytesseract pillow beautifulsoup4 pandas tabulate torch transformers

In [15]:
# import os
# import re
# from pdf2image import convert_from_path
# from PIL import Image
# import pytesseract
# from bs4 import BeautifulSoup
# import pandas as pd
# from tabulate import tabulate
# import shutil
# import torch
# from transformers import LayoutLMv3ForTokenClassification, LayoutLMv3Processor

# # Global constants
# USE_LAYOUTLMV3 = True
# LAYOUTLMV3_MODEL_NAME = "microsoft/layoutlmv3-base-finetuned-sroie"
# LINE_THRESHOLD = 15  # Pixels for line grouping
# AMOUNT_KEYWORDS = ['amount', 'debit', 'credit', 'balance', 'total']
# DATE_KEYWORDS = ['date', 'time']

# # Load LayoutLMv3 model once
# if USE_LAYOUTLMV3:
#     try:
#         layoutlmv3_processor = LayoutLMv3Processor.from_pretrained(LAYOUTLMV3_MODEL_NAME, apply_ocr=False)
#         layoutlmv3_model = LayoutLMv3ForTokenClassification.from_pretrained(LAYOUTLMV3_MODEL_NAME)
#         print("LayoutLMv3 model loaded successfully")
#     except Exception as e:
#         print(f"Error loading LayoutLMv3 model: {e}")
#         USE_LAYOUTLMV3 = False

# def convert_pdf_to_images(pdf_path, output_folder='temp_images'):
#     """Convert PDF pages to images."""
#     os.makedirs(output_folder, exist_ok=True)
#     images = convert_from_path(pdf_path)
#     image_paths = []
#     for i, image in enumerate(images):
#         image_path = os.path.join(output_folder, f'page_{i+1}.jpg')
#         image.save(image_path, 'JPEG')
#         image_paths.append(image_path)
#     return image_paths

# def ocr_image(image_path):
#     """Perform OCR on an image and return hOCR output."""
#     hocr = pytesseract.image_to_pdf_or_hocr(image_path, extension='hocr', config='--psm 6')
#     return hocr

# def parse_hocr_to_words(hocr):
#     """Parse hOCR output to extract words and bounding boxes."""
#     soup = BeautifulSoup(hocr, 'html.parser')
#     words = []
#     for span in soup.find_all('span', class_='ocrx_word'):
#         text = span.get_text().strip()
#         title = span['title']
#         bbox_match = re.search(r'bbox (\d+) (\d+) (\d+) (\d+)', title)
#         if bbox_match:
#             x_min, y_min, x_max, y_max = map(int, bbox_match.groups())
#             words.append({'text': text, 'x_min': x_min, 'y_min': y_min, 'x_max': x_max, 'y_max': y_max})
#     return words

# def group_words_into_lines(words):
#     """Group words into lines based on vertical proximity."""
#     words.sort(key=lambda w: (w['y_min'], w['x_min']))
#     lines = []
#     current_line = []
#     prev_y = None
    
#     for word in words:
#         if prev_y is None or abs(word['y_min'] - prev_y) < LINE_THRESHOLD:
#             current_line.append(word)
#         else:
#             lines.append(current_line)
#             current_line = [word]
#         prev_y = word['y_min']
    
#     if current_line:
#         lines.append(current_line)
    
#     return lines

# def extract_entities_with_layoutlmv3(image_path, words):
#     """Extract entities using LayoutLMv3 model."""
#     if not USE_LAYOUTLMV3:
#         return words
    
#     try:
#         image = Image.open(image_path).convert("RGB")
#         width, height = image.size
        
#         # Normalize bounding boxes to 0-1000 scale
#         normalized_boxes = []
#         for word in words:
#             x0, y0, x1, y1 = word['x_min'], word['y_min'], word['x_max'], word['y_max']
#             normalized_boxes.append([
#                 int(1000 * x0 / width),
#                 int(1000 * y0 / height),
#                 int(1000 * x1 / width),
#                 int(1000 * y1 / height),
#             ])
        
#         tokens = [word['text'] for word in words]
#         encoding = layoutlmv3_processor(
#             image, 
#             tokens, 
#             boxes=normalized_boxes, 
#             return_offsets_mapping=True,
#             return_tensors="pt",
#             truncation=True,
#             padding="max_length",
#             max_length=512
#         )
        
#         # Run model inference
#         with torch.no_grad():
#             outputs = layoutlmv3_model(**encoding)
        
#         # Process predictions
#         predictions = outputs.logits.argmax(-1).squeeze().tolist()
#         offset_mapping = encoding['offset_mapping'].squeeze().tolist()
#         id2label = layoutlmv3_model.config.id2label
        
#         # Map predictions to words
#         previous_word_idx = None
#         for i, word_idx in enumerate(encoding.word_ids(0)):
#             if word_idx is None or word_idx == previous_word_idx:
#                 continue
#             label = id2label[predictions[i]]
#             words[word_idx]['label'] = label
#             previous_word_idx = word_idx
            
#     except Exception as e:
#         print(f"LayoutLMv3 processing error: {e}")
    
#     return words

# def extract_table_from_lines(lines):
#     """Extract table structure from grouped lines."""
#     headers = []
#     data_rows = []
    
#     # Identify header line using common keywords
#     header_line = None
#     for line in lines:
#         line_text = ' '.join(w['text'].lower() for w in line)
#         if any(kw in line_text for kw in DATE_KEYWORDS) and any(kw in line_text for kw in AMOUNT_KEYWORDS):
#             header_line = line
#             break
    
#     if not header_line:
#         return [], []
    
#     # Extract headers
#     headers = [word['text'] for word in header_line]
    
#     # Extract data rows
#     for line in lines[lines.index(header_line) + 1:]:
#         row = {}
#         for i, word in enumerate(line):
#             if i < len(headers):
#                 header = headers[i]
#                 row[header] = row.get(header, '') + ' ' + word['text']
        
#         # Clean row values
#         for key in row:
#             row[key] = row[key].strip()
        
#         if any(row.values()):  # Skip empty rows
#             data_rows.append(row)
    
#     return headers, data_rows

# def clean_extracted_data(data_rows, headers):
#     """Clean and normalize extracted data."""
#     cleaned_data = []
    
#     for row in data_rows:
#         cleaned_row = {}
#         for header in headers:
#             value = row.get(header, '')
            
#             # Clean value based on header type
#             if any(kw in header.lower() for kw in DATE_KEYWORDS):
#                 # Standardize date formats
#                 date_match = re.search(r'\d{2}[-/]\d{2}[-/]\d{2,4}', value)
#                 if date_match:
#                     cleaned_row[header] = date_match.group()
#                 else:
#                     cleaned_row[header] = value
            
#             elif any(kw in header.lower() for kw in AMOUNT_KEYWORDS):
#                 # Extract numerical values
#                 amount_str = re.sub(r'[^\d.-]', '', value)
#                 try:
#                     cleaned_row[header] = float(amount_str) if amount_str else 0.0
#                 except ValueError:
#                     cleaned_row[header] = value
            
#             else:
#                 # Clean descriptive text
#                 cleaned_value = re.sub(r'\s+', ' ', value).strip()
#                 cleaned_row[header] = cleaned_value
        
#         cleaned_data.append(cleaned_row)
    
#     return cleaned_data

# def analyze_pdf(pdf_path):
#     """Analyze PDF and extract transaction data."""
#     image_paths = convert_pdf_to_images(pdf_path)
#     all_data = []
#     headers = []
    
#     for img_path in image_paths:
#         # Get OCR results
#         hocr = ocr_image(img_path)
#         words = parse_hocr_to_words(hocr)
        
#         # Enhanced entity recognition
#         words = extract_entities_with_layoutlmv3(img_path, words)
        
#         # Group words into lines
#         lines = group_words_into_lines(words)
        
#         # Extract table data
#         page_headers, page_data = extract_table_from_lines(lines)
        
#         if page_headers and not headers:
#             headers = page_headers
        
#         if page_data:
#             cleaned_page_data = clean_extracted_data(page_data, headers)
#             all_data.extend(cleaned_page_data)
        
#         # Clean up image file
#         os.remove(img_path)
    
#     # Clean up temporary folder
#     if os.path.exists('temp_images'):
#         shutil.rmtree('temp_images')
    
#     # Create DataFrame
#     if headers and all_data:
#         return pd.DataFrame(all_data, columns=headers)
#     return None

# def display_table(df):
#     """Display DataFrame in table format."""
#     if df is not None and not df.empty:
#         print(tabulate(df, headers=df.columns, tablefmt='psql', showindex=False))
#         print(f"\nTotal Transactions: {len(df)}")
#         print(f"Columns: {', '.join(df.columns)}")
        
#         # Summary statistics
#         amount_cols = [col for col in df.columns if any(kw in col.lower() for kw in AMOUNT_KEYWORDS)]
#         if amount_cols:
#             print("\nAmount Summary:")
#             for col in amount_cols:
#                 if pd.api.types.is_numeric_dtype(df[col]):
#                     print(f"- {col}:")
#                     print(f"  Total: {df[col].sum():.2f}")
#                     print(f"  Avg: {df[col].mean():.2f}")
#                     print(f"  Min: {df[col].min():.2f}")
#                     print(f"  Max: {df[col].max():.2f}")
#     else:
#         print("No transaction data extracted.")

# if __name__ == "__main__":
#     # Update with your PDF path
#     pdf_path = r"C:\Users\HP\Desktop\Bank Project\Balraj\Revoult Bank statement.pdf"
    
#     if not os.path.isfile(pdf_path):
#         print(f"Error: File not found - {pdf_path}")
#     else:
#         print(f"Processing: {pdf_path}")
#         print(f"Using LayoutLMv3: {'Yes' if USE_LAYOUTLMV3 else 'No'}")
        
#         transaction_df = analyze_pdf(pdf_path)
#         display_table(transaction_df)   


import os
import re
from pdf2image import convert_from_path
from PIL import Image
import pytesseract
from bs4 import BeautifulSoup
import pandas as pd
from tabulate import tabulate
import shutil
import torch
from transformers import LayoutLMv3ForTokenClassification, LayoutLMv3Processor

# Global constants
USE_LAYOUTLMV3 = True
LAYOUTLMV3_MODEL_NAME = "microsoft/layoutlmv3-base"
LINE_THRESHOLD = 15  # Pixels for line grouping
AMOUNT_KEYWORDS = ['amount', 'debit', 'credit', 'balance', 'total']
DATE_KEYWORDS = ['date', 'time']

# Load LayoutLMv3 model once
if USE_LAYOUTLMV3:
    try:
        layoutlmv3_processor = LayoutLMv3Processor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)
        layoutlmv3_model = LayoutLMv3ForTokenClassification.from_pretrained("microsoft/layoutlmv3-base")
        print("LayoutLMv3 model loaded successfully")
    except Exception as e:
        print(f"Error loading LayoutLMv3 model: {e}")
        USE_LAYOUTLMV3 = False


def convert_pdf_to_images(pdf_path, output_folder='temp_images'):
    """Convert PDF pages to images."""
    os.makedirs(output_folder, exist_ok=True)
    images = convert_from_path(pdf_path)
    image_paths = []
    for i, image in enumerate(images):
        image_path = os.path.join(output_folder, f'page_{i+1}.jpg')
        image.save(image_path, 'JPEG')
        image_paths.append(image_path)
    return image_paths

def ocr_image(image_path):
    """Perform OCR on an image and return hOCR output."""
    hocr = pytesseract.image_to_pdf_or_hocr(image_path, extension='hocr', config='--psm 6')
    return hocr

def parse_hocr_to_words(hocr):
    """Parse hOCR output to extract words and bounding boxes."""
    soup = BeautifulSoup(hocr, 'html.parser')
    words = []
    for span in soup.find_all('span', class_='ocrx_word'):
        text = span.get_text().strip()
        title = span['title']
        bbox_match = re.search(r'bbox (\d+) (\d+) (\d+) (\d+)', title)
        if bbox_match:
            x_min, y_min, x_max, y_max = map(int, bbox_match.groups())
            words.append({'text': text, 'x_min': x_min, 'y_min': y_min, 'x_max': x_max, 'y_max': y_max})
    return words

def group_words_into_lines(words):
    """Group words into lines based on vertical proximity."""
    words.sort(key=lambda w: (w['y_min'], w['x_min']))
    lines = []
    current_line = []
    prev_y = None
    
    for word in words:
        if prev_y is None or abs(word['y_min'] - prev_y) < LINE_THRESHOLD:
            current_line.append(word)
        else:
            lines.append(current_line)
            current_line = [word]
        prev_y = word['y_min']
    
    if current_line:
        lines.append(current_line)
    
    return lines

def extract_entities_with_layoutlmv3(image_path, words):
    """Extract entities using LayoutLMv3 model."""
    if not USE_LAYOUTLMV3:
        return words
    
    try:
        image = Image.open(image_path).convert("RGB")
        width, height = image.size
        
        # Normalize bounding boxes to 0-1000 scale
        normalized_boxes = []
        for word in words:
            x0, y0, x1, y1 = word['x_min'], word['y_min'], word['x_max'], word['y_max']
            normalized_boxes.append([
                int(1000 * x0 / width),
                int(1000 * y0 / height),
                int(1000 * x1 / width),
                int(1000 * y1 / height),
            ])
        
        tokens = [word['text'] for word in words]
        encoding = layoutlmv3_processor(
            image, 
            tokens, 
            boxes=normalized_boxes, 
            return_offsets_mapping=True,
            return_tensors="pt",
            truncation=True,
            padding="max_length",
            max_length=512
        )
        
        # Run model inference
        with torch.no_grad():
            outputs = layoutlmv3_model(**encoding)
        
        # Process predictions
        predictions = outputs.logits.argmax(-1).squeeze().tolist()
        offset_mapping = encoding['offset_mapping'].squeeze().tolist()
        id2label = layoutlmv3_model.config.id2label
        
        # Map predictions to words
        previous_word_idx = None
        for i, word_idx in enumerate(encoding.word_ids(0)):
            if word_idx is None or word_idx == previous_word_idx:
                continue
            label = id2label[predictions[i]]
            words[word_idx]['label'] = label
            previous_word_idx = word_idx
            
    except Exception as e:
        print(f"LayoutLMv3 processing error: {e}")
    
    return words

def extract_table_from_lines(lines):
    """Extract table structure from grouped lines using clean headers."""
    # Use EXPECTED_HEADERS and COLUMN_THRESHOLD from notebook variables
    expected_headers = EXPECTED_HEADERS
    column_threshold = COLUMN_THRESHOLD

    headers = []
    data_rows = []

    header_line = None
    for line in lines:
        line_text = ' '.join(w['text'].lower() for w in line)
        if all(kw in line_text for kw in ['date', 'description', 'balance']):
            header_line = line
            break

    if not header_line:
        return [], []

    # Sort header words left to right and get X positions
    header_line.sort(key=lambda w: w['x_min'])
    column_positions = [w['x_min'] for w in header_line]

    # Assign expected headers to these positions, trim or pad as needed
    if len(column_positions) > len(expected_headers):
        headers = expected_headers + [f"Extra_{i}" for i in range(len(column_positions) - len(expected_headers))]
    else:
        headers = expected_headers[:len(column_positions)]

    # Now process data rows
    for line in lines[lines.index(header_line) + 1:]:
        row = [''] * len(headers)
        for word in line:
            # Find nearest column by x position within threshold
            distances = [abs(word['x_min'] - col_x) for col_x in column_positions]
            min_dist = min(distances)
            col_idx = distances.index(min_dist)
            # Only assign if within threshold, else skip
            if min_dist <= column_threshold:
                row[col_idx] += word['text'] + ' '
            else:
                # If no close column, skip or append to last column
                row[-1] += word['text'] + ' '

        # Clean whitespace
        row = [val.strip() for val in row]
        if any(row):
            # Only zip up to the number of headers
            data_rows.append(dict(zip(headers, row)))

    return headers, data_rows 

def clean_extracted_data(data_rows, headers):
    """Clean and normalize extracted data."""
    cleaned_data = []
    
    for row in data_rows:
        cleaned_row = {}
        for header in headers:
            value = row.get(header, '')
            
            # Clean value based on header type
            if any(kw in header.lower() for kw in DATE_KEYWORDS):
                # Standardize date formats
                date_match = re.search(r'\d{2}[-/]\d{2}[-/]\d{2,4}', value)
                if date_match:
                    cleaned_row[header] = date_match.group()
                else:
                    cleaned_row[header] = value
            
            elif any(kw in header.lower() for kw in AMOUNT_KEYWORDS):
                # Extract numerical values
                amount_str = re.sub(r'[^\d.-]', '', value)
                try:
                    cleaned_row[header] = float(amount_str) if amount_str else 0.0
                except ValueError:
                    cleaned_row[header] = value
            
            else:
                # Clean descriptive text
                cleaned_value = re.sub(r'\s+', ' ', value).strip()
                cleaned_row[header] = cleaned_value
        
        cleaned_data.append(cleaned_row)
    
    return cleaned_data

def analyze_pdf(pdf_path):
    """Analyze PDF and extract transaction data."""
    image_paths = convert_pdf_to_images(pdf_path)
    all_data = []
    headers = []
    
    for img_path in image_paths:
        # Get OCR results
        hocr = ocr_image(img_path)
        words = parse_hocr_to_words(hocr)
        
        # Enhanced entity recognition
        words = extract_entities_with_layoutlmv3(img_path, words)
        
        # Group words into lines
        lines = group_words_into_lines(words)
        
        # Extract table data
        page_headers, page_data = extract_table_from_lines(lines)
        
        if page_headers and not headers:
            headers = page_headers
        
        if page_data:
            cleaned_page_data = clean_extracted_data(page_data, headers)
            all_data.extend(cleaned_page_data)
        
        # Clean up image file
        os.remove(img_path)
    
    # Clean up temporary folder
    if os.path.exists('temp_images'):
        shutil.rmtree('temp_images')
    
    # Create DataFrame
    if headers and all_data:
        return pd.DataFrame(all_data, columns=headers)
    return None

def display_table(df):
    """Display DataFrame in table format."""
    if df is not None and not df.empty:
        print(tabulate(df, headers=df.columns, tablefmt='psql', showindex=False))
        print(f"\nTotal Transactions: {len(df)}")
        print(f"Columns: {', '.join(df.columns)}")
        
        # Summary statistics
        amount_cols = [col for col in df.columns if any(kw in col.lower() for kw in AMOUNT_KEYWORDS)]
        if amount_cols:
            print("\nAmount Summary:")
            for col in amount_cols:
                if pd.api.types.is_numeric_dtype(df[col]):
                    print(f"- {col}:")
                    print(f"  Total: {df[col].sum():.2f}")
                    print(f"  Avg: {df[col].mean():.2f}")
                    print(f"  Min: {df[col].min():.2f}")
                    print(f"  Max: {df[col].max():.2f}")
    else:
        print("No transaction data extracted.")

if __name__ == "__main__":
    # Update with your PDF path
    pdf_path = r"C:\Users\HP\Desktop\Bank Project\Balraj\Revoult Bank statement.pdf"
    
    if not os.path.isfile(pdf_path):
        print(f"Error: File not found - {pdf_path}")
    else:
        print(f"Processing: {pdf_path}")
        print(f"Using LayoutLMv3: {'Yes' if USE_LAYOUTLMV3 else 'No'}")
        
        transaction_df = analyze_pdf(pdf_path)
        display_table(transaction_df)  



Some weights of LayoutLMv3ForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


LayoutLMv3 model loaded successfully
Processing: C:\Users\HP\Desktop\Bank Project\Balraj\Revoult Bank statement.pdf
Using LayoutLMv3: Yes
LayoutLMv3 processing error: LayoutLMv3ForTokenClassification.forward() got an unexpected keyword argument 'offset_mapping'
+------------------------------------------------------------+-----------------------------------------------+-------------------------------------------+-----------------------------------------+-------------------------------------+-----------------+-----------+------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| date                                                       | value date                                    | description                               | money out                               | money in                            | bal

In [30]:
import pdfplumber
import pandas as pd
import os
import logging
import re
from tabulate import tabulate
import torch
from transformers import LayoutLMv3ForTokenClassification, LayoutLMv3Processor
from PIL import Image
import numpy as np
import tempfile
from pdf2image import convert_from_path
import time  # Added for retry mechanism

# Enable logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 🔧 Hardcode your PDF file path here
PDF_PATH = r"C:\Users\HP\Desktop\Bank Project\Balraj\Revoult Bank statement.pdf"

# Optional: Export to CSV — set to None to disable
EXPORT_CSV_PATH = r"C:\Users\HP\Desktop\Bank Project\output.csv"  # or None to skip exporting

# LayoutLMv3 Configuration
LAYOUTLMV3_MODEL_NAME = "microsoft/layoutlmv3-base-finetoned-sroie"
AMOUNT_KEYWORDS = ['amount', 'debit', 'credit', 'balance', 'total', 'amt', 'value']
DATE_KEYWORDS = ['date', 'time', 'transaction date', 'posted date']
DESCRIPTION_KEYWORDS = ['description', 'merchant', 'details', 'narration', 'transaction', 'particulars']

# Load LayoutLMv3 model
try:
    layoutlmv3_processor = LayoutLMv3Processor.from_pretrained(LAYOUTLMV3_MODEL_NAME, apply_ocr=False)
    layoutlmv3_model = LayoutLMv3ForTokenClassification.from_pretrained(LAYOUTLMV3_MODEL_NAME)
    logging.info("LayoutLMv3 model loaded successfully")
    USE_LAYOUTLMV3 = True
except Exception as e:
    logging.warning(f"Error loading LayoutLMv3 model: {e}. Using fallback method.")
    USE_LAYOUTLMV3 = False

def make_unique(headers):
    seen = {}
    unique = []
    for i, col in enumerate(headers):
        col = col.strip() if col else f"Column_{i}"
        if col in seen:
            seen[col] += 1
            unique.append(f"{col}_{seen[col]}")
        else:
            seen[col] = 0
            unique.append(col)
    return unique

def extract_entities_with_layoutlmv3(page_image, words):
    """Extract entities using LayoutLMv3 model."""
    if not USE_LAYOUTLMV3:
        return words
    
    try:
        width, height = page_image.size
        
        # Normalize bounding boxes to 0-1000 scale
        normalized_boxes = []
        for word in words:
            x0, y0, x1, y1 = word['x_min'], word['y_min'], word['x_max'], word['y_max']
            normalized_boxes.append([
                int(1000 * x0 / width),
                int(1000 * y0 / height),
                int(1000 * x1 / width),
                int(1000 * y1 / height),
            ])
        
        tokens = [word['text'] for word in words]
        encoding = layoutlmv3_processor(
            page_image, 
            tokens, 
            boxes=normalized_boxes, 
            return_offsets_mapping=True,
            return_tensors="pt",
            truncation=True,
            padding="max_length",
            max_length=512
        )
        
        # Run model inference
        with torch.no_grad():
            outputs = layoutlmv3_model(**encoding)
        
        # Process predictions
        predictions = outputs.logits.argmax(-1).squeeze().tolist()
        id2label = layoutlmv3_model.config.id2label
        
        # Map predictions to words
        previous_word_idx = None
        for i, word_idx in enumerate(encoding.word_ids(0)):
            if word_idx is None or word_idx == previous_word_idx:
                continue
            label = id2label[predictions[i]]
            words[word_idx]['label'] = label
            previous_word_idx = word_idx
            
    except Exception as e:
        logging.error(f"LayoutLMv3 processing error: {e}")
    
    return words

def validate_and_correct_headers(headers, page_labeled_words):
    """Use LayoutLMv3 to validate and correct headers."""
    if not headers or not page_labeled_words or not USE_LAYOUTLMV3:
        return headers
    
    # Identify header entities
    header_entities = []
    for word in page_labeled_words:
        if word.get('label', '') in ['B-HEADER', 'I-HEADER']:
            header_entities.append(word['text'])
    
    # If we found better header candidates, use them
    if header_entities:
        logging.info(f"LayoutLMv3 identified header entities: {header_entities}")
        return header_entities
    
    return headers

def validate_and_correct_row(row, headers, page_labeled_words):
    """Use LayoutLMv3 to validate and correct row data."""
    if not page_labeled_words or not USE_LAYOUTLMV3:
        return row
    
    corrected_row = {}
    for header in headers:
        header_lower = header.lower()
        cell_value = row.get(header, '')
        
        # Look for matching entities
        entity_values = []
        for word in page_labeled_words:
            label = word.get('label', '')
            text = word['text']
            
            if any(kw in header_lower for kw in DATE_KEYWORDS) and label in ['B-DATE', 'I-DATE']:
                entity_values.append(text)
            elif any(kw in header_lower for kw in AMOUNT_KEYWORDS) and label in ['B-AMOUNT', 'I-AMOUNT']:
                entity_values.append(text)
            elif label in ['B-COMPANY', 'I-COMPANY']:
                entity_values.append(text)
        
        # Use entity values if found
        if entity_values:
            corrected_value = ' '.join(entity_values)
            if corrected_value.strip():
                corrected_row[header] = corrected_value
                continue
        
        # Fallback to original value
        corrected_row[header] = cell_value
    
    return corrected_row

def clean_value(header, value):
    """Clean value based on header type."""
    header_lower = header.lower()
    
    if any(kw in header_lower for kw in DATE_KEYWORDS):
        # Standardize date formats
        date_match = re.search(r'\d{2}[-/]\d{2}[-/]\d{2,4}', value)
        if date_match:
            return date_match.group()
        return value
    
    elif any(kw in header_lower for kw in AMOUNT_KEYWORDS):
        # Extract numerical values
        amount_str = re.sub(r'[^\d.-]', '', value)
        try:
            return float(amount_str) if amount_str else 0.0
        except ValueError:
            return value
    
    # Clean descriptive text
    return re.sub(r'\s+', ' ', value).strip()

def safe_cleanup(temp_dir, max_retries=3, delay=1):
    """Safely clean up temporary directory with retries."""
    for i in range(max_retries):
        try:
            # Close any open file handles
            import gc
            gc.collect()
            
            # Try to remove the directory
            for root, dirs, files in os.walk(temp_dir, topdown=False):
                for name in files:
                    file_path = os.path.join(root, name)
                    try:
                        os.unlink(file_path)
                    except Exception as e:
                        logging.warning(f"Could not delete {file_path}: {e}")
                for name in dirs:
                    os.rmdir(os.path.join(root, name))
            os.rmdir(temp_dir)
            return True
        except Exception as e:
            logging.warning(f"Cleanup attempt {i+1} failed: {e}")
            time.sleep(delay)
    logging.error(f"Failed to cleanup temporary directory after {max_retries} attempts")
    return False

def parse_bank_statement(path):
    all_dataframes = []
    master_headers = None
    page_image_paths = {}
    temp_image_dir = tempfile.mkdtemp()
    logging.info(f"Created temporary image directory: {temp_image_dir}")

    try:
        # Convert PDF to images and save to files
        images = convert_from_path(path, output_folder=temp_image_dir, fmt='jpeg')
        for i, image in enumerate(images):
            image_path = os.path.join(temp_image_dir, f"page_{i+1}.jpg")
            image.save(image_path, 'JPEG')
            image.close()  # Close the image to release the file handle
            page_image_paths[i+1] = image_path
        logging.info(f"Converted {len(images)} pages to images")
    except Exception as e:
        logging.error(f"PDF to image conversion failed: {e}")
        safe_cleanup(temp_image_dir)
        return pd.DataFrame()

    with pdfplumber.open(path) as pdf:
        for page_num, page in enumerate(pdf.pages, 1):
            page_labeled_words = []
            image_path = page_image_paths.get(page_num)
            
            if image_path and os.path.exists(image_path):
                try:
                    # Open image only when needed
                    with Image.open(image_path) as page_image:
                        # Extract words using pdfplumber
                        words = page.extract_words(extra_attrs=["x0", "top", "x1", "bottom"])
                        page_words = []
                        for word in words:
                            page_words.append({
                                'text': word['text'],
                                'x_min': word['x0'],
                                'y_min': word['top'],
                                'x_max': word['x1'],
                                'y_max': word['bottom']
                            })
                        
                        # Process with LayoutLMv3
                        if USE_LAYOUTLMV3:
                            page_labeled_words = extract_entities_with_layoutlmv3(page_image, page_words)
                        else:
                            page_labeled_words = page_words
                except Exception as e:
                    logging.error(f"Error processing image for page {page_num}: {e}")
            
            tables = page.extract_tables()
            logging.info(f"📄 Page {page_num}: Found {len(tables)} tables")

            for idx, table in enumerate(tables):
                if not table or len(table) < 2:
                    continue  # Skip empty or too-small tables

                headers = table[0]
                rows = table[1:]

                # Validate and correct headers with LayoutLMv3
                headers = validate_and_correct_headers(headers, page_labeled_words)
                
                # If headers are valid and not repeated junk rows, use them
                cleaned_headers = [h.strip() if h else f"Column_{i}" for i, h in enumerate(headers)]
                unique_headers = make_unique(cleaned_headers)

                # Save headers from the first valid table
                if master_headers is None and len(set(unique_headers)) == len(unique_headers):
                    master_headers = unique_headers
                elif master_headers and len(headers) == len(master_headers):
                    # Assume headers are missing; treat entire table as data
                    rows = table  # include all rows (no header)
                    cleaned_headers = master_headers
                else:
                    continue  # Skip inconsistent tables

                # Clean and validate rows
                cleaned_rows = []
                for row in rows:
                    # Create dictionary for the row
                    row_dict = {}
                    for i, header in enumerate(cleaned_headers):
                        if i < len(row):
                            row_dict[header] = row[i].strip() if row[i] else ''
                        else:
                            row_dict[header] = ''
                    
                    # Validate and correct with LayoutLMv3
                    row_dict = validate_and_correct_row(row_dict, cleaned_headers, page_labeled_words)
                    
                    # Clean values
                    cleaned_row = {}
                    for header, value in row_dict.items():
                        cleaned_row[header] = clean_value(header, value)
                    
                    # Only include rows with at least one non-empty value
                    if any(cleaned_row.values()):
                        cleaned_rows.append(cleaned_row)

                if not cleaned_rows:
                    continue

                try:
                    df = pd.DataFrame(cleaned_rows, columns=cleaned_headers)
                    all_dataframes.append(df)
                    logging.info(f"✅ Page {page_num} Table {idx + 1}: Parsed {len(df)} rows")
                except Exception as e:
                    logging.warning(f"⚠️ Skipping malformed table on page {page_num}: {e}")

    # Clean up temporary files
    safe_cleanup(temp_image_dir)

    if not all_dataframes:
        return pd.DataFrame()

    return pd.concat(all_dataframes, ignore_index=True)

def main():
    if not os.path.isfile(PDF_PATH):
        print(f"❌ File does not exist: {PDF_PATH}")
        return

    print(f"\n📄 Processing file: {PDF_PATH}")
    print(f"Using LayoutLMv3: {'Yes' if USE_LAYOUTLMV3 else 'No'}")
    
    df = parse_bank_statement(PDF_PATH)

    if df.empty:
        print("⚠️ No transaction data found.")
    else:
        print("\n✅ Extracted Data:\n")
        print(tabulate(df, headers='keys', tablefmt='grid', showindex=False))

        if EXPORT_CSV_PATH:
            try:
                df.to_csv(EXPORT_CSV_PATH, index=False)
                print(f"\n📁 Exported to: {EXPORT_CSV_PATH}")
            except Exception as e:
                print(f"❌ Failed to export CSV: {e}")


if __name__ == "__main__":
    main()

2025-08-07 17:40:35,368 - WARNING - Error loading LayoutLMv3 model: microsoft/layoutlmv3-base-finetoned-sroie is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`. Using fallback method.
2025-08-07 17:40:35,381 - INFO - Created temporary image directory: C:\Users\HP\AppData\Local\Temp\tmpw_t2dxzg



📄 Processing file: C:\Users\HP\Desktop\Bank Project\Balraj\Revoult Bank statement.pdf
Using LayoutLMv3: No


2025-08-07 17:40:35,974 - INFO - Converted 1 pages to images


⚠️ No transaction data found.


In [37]:
# Import necessary libraries
import torch
from transformers import LayoutLMv3Processor, LayoutLMv3ForTokenClassification
from PIL import Image
import pytesseract
import pandas as pd
import pdf2image

# Function to convert PDF to images
def pdf_to_images(pdf_path):
    images = pdf2image.convert_from_path(pdf_path)
    return images

# Function to extract text from images using Tesseract
def extract_text_from_image(image):
    return pytesseract.image_to_string(image)

# Function to process images and extract data using LayoutLMv3
def extract_data_with_layoutlm(images):
    processor = LayoutLMv3Processor.from_pretrained("microsoft/layoutlmv3-base")
    model = LayoutLMv3ForTokenClassification.from_pretrained("microsoft/layoutlmv3-base")
    
    data = []
    
    for image in images:
        # Convert image to the required format
        words = extract_text_from_image(image)
        encoding = processor(image, words, return_tensors="pt", truncation=True, padding="max_length", max_length=512)
        
        # Get predictions
        with torch.no_grad():
            outputs = model(**encoding)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=2)
        
        # Collect data
        for word, prediction in zip(words.split(), predictions[0].numpy()):
            data.append((word, prediction))
    
    return data

# Function to create a DataFrame from extracted data
def create_dataframe(data):
    df = pd.DataFrame(data, columns=["Word", "Prediction"])
    return df

# Main execution
if __name__ == "__main__":
    pdf_path = r"C:\Users\HP\Desktop\Bank Project\Balraj\Barclays_uk_bank_statement.pdf"  # Path to your PDF file
    images = pdf_to_images(pdf_path)
    extracted_data = extract_data_with_layoutlm(images)
    df = create_dataframe(extracted_data)
    
    # Save DataFrame to CSV
    df.to_csv("extracted_bank_statement_data.csv", index=False)
    print("Data extraction complete. Data saved to 'extracted_bank_statement_data.csv'.")


Some weights of LayoutLMv3ForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Data extraction complete. Data saved to 'extracted_bank_statement_data.csv'.


In [6]:
import os
import re
from pdf2image import convert_from_path
from PIL import Image
import pytesseract
from bs4 import BeautifulSoup
import pandas as pd
from tabulate import tabulate
import shutil
import torch
from transformers import LayoutLMv3ForTokenClassification, LayoutLMv3Processor

# Global constants
USE_LAYOUTLMV3 = True
LAYOUTLMV3_MODEL_NAME = "microsoft/layoutlmv3-base-finetuned-sroie"
LINE_THRESHOLD = 15  # Pixels for line grouping
AMOUNT_KEYWORDS = ['amount', 'debit', 'credit', 'balance', 'total']
DATE_KEYWORDS = ['date', 'time']

# Load LayoutLMv3 model once
if USE_LAYOUTLMV3:
    try:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {device}")
        
        layoutlmv3_processor = LayoutLMv3Processor.from_pretrained(LAYOUTLMV3_MODEL_NAME, apply_ocr=False)
        layoutlmv3_model = LayoutLMv3ForTokenClassification.from_pretrained(LAYOUTLMV3_MODEL_NAME).to(device)
        print("LayoutLMv3 model loaded successfully")
    except Exception as e:
        print(f"Error loading LayoutLMv3 model: {e}")
        USE_LAYOUTLMV3 = False

def convert_pdf_to_images(pdf_path, output_folder='temp_images'):
    """Convert PDF pages to images."""
    os.makedirs(output_folder, exist_ok=True)
    try:
        images = convert_from_path(pdf_path)
        image_paths = []
        for i, image in enumerate(images):
            image_path = os.path.join(output_folder, f'page_{i+1}.jpg')
            image.save(image_path, 'JPEG')
            image_paths.append(image_path)
        return image_paths
    except Exception as e:
        print(f"Error converting PDF to images: {e}")
        return []

def ocr_image(image_path):
    """Perform OCR on an image and return hOCR output."""
    try:
        hocr = pytesseract.image_to_pdf_or_hocr(image_path, extension='hocr', config='--psm 6')
        return hocr
    except Exception as e:
        print(f"OCR failed for {image_path}: {e}")
        return None

def parse_hocr_to_words(hocr):
    """Parse hOCR output to extract words and bounding boxes."""
    if not hocr:
        return []
    
    try:
        soup = BeautifulSoup(hocr, 'html.parser')
        words = []
        for span in soup.find_all('span', class_='ocrx_word'):
            text = span.get_text().strip()
            title = span.get('title', '')
            bbox_match = re.search(r'bbox (\d+) (\d+) (\d+) (\d+)', title)
            if bbox_match and text:
                x_min, y_min, x_max, y_max = map(int, bbox_match.groups())
                words.append({'text': text, 'x_min': x_min, 'y_min': y_min, 'x_max': x_max, 'y_max': y_max})
        return words
    except Exception as e:
        print(f"Error parsing hOCR: {e}")
        return []

def group_words_into_lines(words):
    """Group words into lines based on vertical proximity."""
    if not words:
        return []
    
    words.sort(key=lambda w: (w['y_min'], w['x_min']))
    lines = []
    current_line = []
    prev_y = None
    
    for word in words:
        if prev_y is None or abs(word['y_min'] - prev_y) < LINE_THRESHOLD:
            current_line.append(word)
        else:
            if current_line:
                lines.append(current_line)
            current_line = [word]
        prev_y = word['y_min']
    
    if current_line:
        lines.append(current_line)
    
    return lines

def extract_entities_with_layoutlmv3(image_path, words):
    """Extract entities using LayoutLMv3 model."""
    if not USE_LAYOUTLMV3 or not words:
        return words
    
    try:
        image = Image.open(image_path).convert("RGB")
        width, height = image.size
        
        # Normalize bounding boxes to 0-1000 scale
        normalized_boxes = []
        for word in words:
            x0, y0, x1, y1 = word['x_min'], word['y_min'], word['x_max'], word['y_max']
            normalized_boxes.append([
                max(0, int(1000 * x0 / width)),
                max(0, int(1000 * y0 / height)),
                min(1000, int(1000 * x1 / width)),
                min(1000, int(1000 * y1 / height)),
            ])
        
        tokens = [word['text'] for word in words]
        encoding = layoutlmv3_processor(
            image, 
            tokens, 
            boxes=normalized_boxes, 
            return_offsets_mapping=True,
            return_tensors="pt",
            truncation=True,
            padding="max_length",
            max_length=512
        )
        
        # Move to device
        encoding = {k: v.to(device) for k, v in encoding.items()}
        
        # Run model inference
        with torch.no_grad():
            outputs = layoutlmv3_model(**encoding)
        
        # Process predictions
        predictions = outputs.logits.argmax(-1).squeeze().tolist()
        offset_mapping = encoding['offset_mapping'].squeeze().tolist()
        id2label = layoutlmv3_model.config.id2label
        
        # Map predictions to words
        previous_word_idx = None
        for i, word_idx in enumerate(encoding.word_ids(0)):
            if word_idx is None or word_idx == previous_word_idx:
                continue
            label = id2label[predictions[i]]
            words[word_idx]['label'] = label
            previous_word_idx = word_idx
            
        print(f"LayoutLMv3 processed {len(words)} words")
    except Exception as e:
        print(f"LayoutLMv3 processing error: {e}")
    
    return words

def extract_table_from_lines(lines):
    """Extract table structure from grouped lines using grid-based approach."""
    if not lines:
        return [], []
    
    # Find header line index using keywords
    header_index = None
    for idx, line in enumerate(lines):
        line_text = ' '.join(w['text'].lower() for w in line)
        if any(kw in line_text for kw in DATE_KEYWORDS) and any(kw in line_text for kw in AMOUNT_KEYWORDS):
            header_index = idx
            break
    
    # Fallback: find first line with multiple columns
    if header_index is None:
        for idx, line in enumerate(lines):
            if len(line) >= 3:  # At least 3 columns
                header_index = idx
                break
    
    if header_index is None:
        return [], []
    
    header_line = lines[header_index]
    # Sort header words by x-position
    header_line_sorted = sorted(header_line, key=lambda w: w['x_min'])
    headers = [word['text'] for word in header_line_sorted]
    
    # Calculate column boundaries using header positions
    boundaries = []
    for i in range(len(header_line_sorted) - 1):
        boundary = (header_line_sorted[i]['x_max'] + header_line_sorted[i+1]['x_min']) / 2
        boundaries.append(boundary)
    
    # Add left and right boundaries
    boundaries = [0] + boundaries + [100000]  # Large number for right boundary
    
    data_rows = []
    # Process subsequent lines
    for line in lines[header_index+1:]:
        # Skip empty lines
        if not line:
            continue
            
        # Sort words by x-position
        sorted_line = sorted(line, key=lambda w: w['x_min'])
        row = [''] * len(headers)
        
        for word in sorted_line:
            center = (word['x_min'] + word['x_max']) / 2
            # Find matching column
            for col_idx in range(len(boundaries)-1):
                if boundaries[col_idx] <= center < boundaries[col_idx+1]:
                    # Append text to column
                    if row[col_idx]:
                        row[col_idx] += ' ' + word['text']
                    else:
                        row[col_idx] = word['text']
                    break
        
        # Convert to dictionary
        row_dict = {header: value.strip() for header, value in zip(headers, row)}
        data_rows.append(row_dict)
    
    return headers, data_rows

def clean_extracted_data(data_rows, headers):
    """Clean and normalize extracted data."""
    cleaned_data = []
    
    for row in data_rows:
        cleaned_row = {}
        for header in headers:
            value = row.get(header, '')
            
            # Clean value based on header type
            if any(kw in header.lower() for kw in DATE_KEYWORDS):
                # Standardize date formats
                date_match = re.search(r'\d{2}[-/]\d{2}[-/]\d{2,4}', value)
                if date_match:
                    cleaned_row[header] = date_match.group()
                else:
                    cleaned_row[header] = value
            
            elif any(kw in header.lower() for kw in AMOUNT_KEYWORDS):
                # Extract numerical values with currency symbols
                amount_str = re.sub(r'[^\d.,-]', '', value)
                # Handle thousand separators
                amount_str = amount_str.replace(',', '').replace(' ', '')
                try:
                    cleaned_row[header] = float(amount_str) if amount_str else 0.0
                except ValueError:
                    cleaned_row[header] = value
            
            else:
                # Clean descriptive text
                cleaned_value = re.sub(r'\s+', ' ', value).strip()
                cleaned_row[header] = cleaned_value
        
        cleaned_data.append(cleaned_row)
    
    return cleaned_data

def analyze_pdf(pdf_path):
    """Analyze PDF and extract transaction data."""
    try:
        image_paths = convert_pdf_to_images(pdf_path)
        if not image_paths:
            print("No images generated from PDF")
            return None
            
        all_data = []
        headers = []
        
        for img_path in image_paths:
            # Get OCR results
            hocr = ocr_image(img_path)
            if not hocr:
                continue
                
            words = parse_hocr_to_words(hocr)
            if not words:
                print(f"No words extracted from {img_path}")
                continue
                
            # Enhanced entity recognition
            words = extract_entities_with_layoutlmv3(img_path, words)
            
            # Group words into lines
            lines = group_words_into_lines(words)
            if not lines:
                print(f"No lines formed from {img_path}")
                continue
                
            # Extract table data
            page_headers, page_data = extract_table_from_lines(lines)
            if not page_headers or not page_data:
                print(f"No table data extracted from {img_path}")
                continue
                
            if not headers:
                headers = page_headers
                
            cleaned_page_data = clean_extracted_data(page_data, headers)
            all_data.extend(cleaned_page_data)
        
        # Create DataFrame
        if headers and all_data:
            return pd.DataFrame(all_data, columns=headers)
        return None
    finally:
        # Clean up temporary files
        if os.path.exists('temp_images'):
            shutil.rmtree('temp_images')

def display_table(df):
    """Display DataFrame in table format."""
    if df is not None and not df.empty:
        print(tabulate(df, headers=df.columns, tablefmt='psql', showindex=False))
        print(f"\nTotal Transactions: {len(df)}")
        print(f"Columns: {', '.join(df.columns)}")
        
        # Summary statistics
        amount_cols = [col for col in df.columns if any(kw in col.lower() for kw in AMOUNT_KEYWORDS)]
        if amount_cols:
            print("\nAmount Summary:")
            for col in amount_cols:
                if pd.api.types.is_numeric_dtype(df[col]):
                    print(f"- {col}:")
                    print(f"  Total: {df[col].sum():.2f}")
                    print(f"  Avg: {df[col].mean():.2f}")
                    print(f"  Min: {df[col].min():.2f}")
                    print(f"  Max: {df[col].max():.2f}")
    else:
        print("No transaction data extracted.")

if __name__ == "__main__":
    # Update with your PDF path
    pdf_path = r"C:\Users\HP\Desktop\Bank Project\Balraj\Revoult Bank statement.pdf"
    
    if not os.path.isfile(pdf_path):
        print(f"Error: File not found - {pdf_path}")
    else:
        print(f"Processing: {pdf_path}")
        print(f"Using LayoutLMv3: {'No' if USE_LAYOUTLMV3 else 'Yes'}")
        
        transaction_df = analyze_pdf(pdf_path)
        display_table(transaction_df)

Using device: cpu
Error loading LayoutLMv3 model: microsoft/layoutlmv3-base-finetuned-sroie is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`
Processing: C:\Users\HP\Desktop\Bank Project\Balraj\Revoult Bank statement.pdf
Using LayoutLMv3: Yes
+------------------------------------------------+----------------------------------+------------------------------------------------+----------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------+----------------------+---------------------------------------------------+-----------+
| Date                 